In [1]:
import numpy as np
import keras 
import tensorflow as tf

In [2]:


# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

# normalize
x_train = x_train.reshape(60000, -1).astype(np.float32) / 255.
x_test = x_test.reshape(10000, -1).astype(np.float32) / 255.

# to one-hot
y_train = tf.one_hot(y_train, 10, 1., 0.).numpy()
y_test = tf.one_hot(y_test, 10, 1., 0.).numpy()

# Define neural network inputnoutput
input_size = x_train.shape[1]
output_size = y_train.shape[1]


print(x_train.shape,
x_test.shape,
y_train.shape,
y_test.shape)

(60000, 784) (10000, 784) (60000, 10) (10000, 10)


In [3]:
# this is just for working with notebooks and classes outside the scope
import importlib
import NeuralNetwork
import Activations
importlib.reload(NeuralNetwork)
importlib.reload(Activations)
NeuralNetwork = NeuralNetwork.NeuralNetwork
########################################################################


# Create and train the neural network
neural_network = NeuralNetwork(input_size, output_size)

256 784
256 256
10 256


In [7]:
neural_network.train(x_train, y_train, learning_rate=0.001, num_epochs=10, batch_size=64)

# Make predictions on the test set
y_pred = neural_network.predict(x_test)

# Calculate accuracy
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print(f"Accuracy: {accuracy * 100}%")

Epoch 1/1, Loss: 0.0003530288528405445
Accuracy: 97.94%


In [202]:
def stochastic_rounding(x):

    x_int = x.astype(np.int32)
    x_frac = np.abs(x - x_int)
    rng = np.random.random(x_int.shape)

    # positive
    rounded_pos = np.where(x_frac > rng, x_int + 1, x_int)

    # negative
    rounded_neg = np.where(x_frac > rng, x_int - 1, x_int)

    rounded = np.where(x < 0, rounded_neg, rounded_pos)
    return rounded

x = np.random.normal(0, 5, (5)) 
print("float ", x)
rx = 0
it = 100
for _ in range(it):
    rx += stochastic_rounding(x)

rx = rx.astype(np.float32) / it
print("stchr0", stochastic_rounding(x))
print("stchr1", rx)
print("round ", np.around(x).astype(np.int32))

float  [-13.29367653  -3.53350411   0.60825117  -0.44525403   4.07995509]
stchr0 [-13  -3   1   0   4]
stchr1 [-13.31  -3.46   0.67  -0.45   4.08]
round  [-13  -4   1   0   4]


In [206]:

def quantize(x, round_stoch = True):
    """ exponentiation and quantization function """

    # just to avoid numerical problems
    eps = 1e-8

    s = np.sign(x)

    abs_x = np.abs(x)

    cliped_abs_x = np.where(abs_x < eps, eps, abs_x)
    cliped_abs_x = np.where(cliped_abs_x > 1, 1, cliped_abs_x)

    # gets the exponent with base 2
    exp = np.log2(cliped_abs_x)

    # round to nearest and cast to int (use stochastic rounding)
    if round_stoch:
        round_exp = stochastic_rounding(exp)
    else:
        round_exp = (np.round(exp)).astype(np.int32)

    # clip to the limit of sx4
    clip_exp = np.where(round_exp < -7, -7, round_exp)

    # convert to float32 again
    qx = s * np.power(2., clip_exp)
    return qx


for _ in range(10):
    x = np.random.normal(0, 0.2, (5)) 
    print("float ", x)
    print("quant0", quantize(x))
    print("quant1", quantize(x))
    print("quant2", quantize(x))    
    print("-------")

float  [ 0.13540721  0.01973048  0.09533211  0.09371898 -0.04145135]
quant0 [ 0.125     0.015625  0.0625    0.0625   -0.03125 ]
quant1 [ 0.125     0.015625  0.125     0.0625   -0.03125 ]
quant2 [ 0.125     0.015625  0.0625    0.0625   -0.03125 ]
-------
float  [ 0.22523366  0.18175557  0.07043789  0.01502677 -0.51716829]
quant0 [ 0.25      0.125     0.0625    0.015625 -0.5     ]
quant1 [ 0.25      0.125     0.0625    0.015625 -0.5     ]
quant2 [ 0.25      0.125     0.0625    0.015625 -0.5     ]
-------
float  [-0.10867882  0.13076258 -0.07715446 -0.01135837 -0.29902023]
quant0 [-0.125     0.125    -0.0625   -0.015625 -0.25    ]
quant1 [-0.125     0.125    -0.0625   -0.015625 -0.5     ]
quant2 [-0.0625    0.125    -0.125    -0.015625 -0.25    ]
-------
float  [0.33149954 0.03191055 0.29109754 0.01527782 0.10594643]
quant0 [0.5      0.03125  0.25     0.015625 0.0625  ]
quant1 [0.25     0.0625   0.5      0.015625 0.125   ]
quant2 [0.25     0.03125  0.5      0.015625 0.0625  ]
-------
floa